# Azure Function to build Storage Accounts

We have a table that contains the details of the required storage accounts - let's have Azure SQL build them out.

Requirements:
- The Azure SQL DB table should be the 'system of record' - so the single source of truth
- Adding rows should create a storage account
- Deleting rows should remove the storage account
- Modifying rows?

Questions remain in this example:
- How does the data get here?
    - Maybe another API to insert data - from change management system
    - A front end web application that lets users request storage accounts
    - ??



## PreReqs
- Create the Azure Function to do the work
- Configure Authentication
    - There are options - best option is Managed Identity
        - Enable Managed Identity for Azure SQL Database
            - This creates an app id to use for authentication
        - Enabled Azure AD authentication for Azure Function

## Demo

1. Create a database master key if one doesn't exist - with a proper strong password

In [4]:
-- create a database master key
IF NOT EXISTS(SELECT * FROM sys.symmetric_keys WHERE name = '##MS_DatabaseMasterKey##')
BEGIN
    create master key encryption by password = '*********'
END

Commands completed successfully.

Total execution time: 00:00:00.027

2. Store the key for the function as a `DATABSAE SCOPED CREDENTIAL`

(Could also be the app_id for the SQL Server managed identity)

In [5]:
IF NOT EXISTS (SELECT 1 FROM sys.database_credentials WHERE name = 'https://psconfeu2023.azurewebsites.net/api/NewStorageAcct')
BEGIN
    -- create a database scoped credential for managed identity (or Request Headers or Query String)
    CREATE DATABASE SCOPED CREDENTIAL [https://psconfeu2023.azurewebsites.net/api/NewStorageAcct]
    WITH IDENTITY = 'HTTPEndpointHeaders', SECRET = '{"Ocp-Apim-Subscription-Key":"*********"}';
END

Commands completed successfully.

Total execution time: 00:00:00.024

3. Call `sp_invoke_external_rest_endpoint` to create a storage account

In [7]:
declare @name varchar(24) = 'psconf2023stg8',
	    @url varchar(500)

set @url = 'https://psconfeu2023.azurewebsites.net/api/NewStorageAcct?name=' + LOWER(@Name)

declare @ret as int, @response as nvarchar(max);

exec @ret = sp_invoke_external_rest_endpoint 
	@method = 'GET',
	@url = @url,
    @timeout = 230, -- max value which is 3.8 mins
    @credential = [https://psconfeu2023.azurewebsites.net/api/NewStorageAcct],
 	@response = @response output;
	
select @ret as ReturnCode, @response as Response;

GO

(1 row affected)

Total execution time: 00:00:21.446

ReturnCode,Response
0,"{""response"":{""status"":{""http"":{""code"":200,""description"":""OK""}},""headers"":{""Date"":""Sun, 11 Jun 2023 13:17:22 GMT"",""Transfer-Encoding"":""chunked"",""Content-Type"":""application\/json; charset=utf-8"",""Request-Context"":""appId=cid-v1:13446fe7-e7d5-484b-8f38-7678af24ebc6"",""Strict-Transport-Security"":""max-age=31536000; includeSubDomains""}},""result"":{ ""StorageAccountName"": ""psconf2023stg8"", ""ProvisioningState"": ""Succeeded"", ""CreationTime"": ""2023-06-11T13:17:02.3365236Z"", ""Tags"": { ""CreatedBy"": ""AzFunc"" } }}"
